<a href="https://colab.research.google.com/github/R-aryan/Text-Generation-using-RNN/blob/master/Text_generation_using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Generation with Neural Networks

In [0]:
train_path='/content/drive/My Drive/Custom_Modelling/shakespeare.txt'

In [3]:
# GOOGLE COLLAB USERS ONLY
%tensorflow_version 2.x

TensorFlow 2.x selected.


In this notebook we will create a network that can generate text, here we show it being done character by character. Very awesome write up on this here: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

We organized the process into "steps" so you can easily follow along with your own data sets.